In [2]:
# Packages
import h5py    
import numpy as np    
import matplotlib.pyplot as plt
from scipy.constants import pi
from scipy.optimize import curve_fit as fit
import func as func

# To quit warnings for showing up in cell outputs:
import warnings
warnings.filterwarnings('ignore')

### v=0 final snapshot fitting

In [3]:
# Directory with data:
datadir = "v0/snaps/"



# Import data:
f = h5py.File(datadir+"snapshot_010.hdf5", "r")
group = f["PartType1"]
data = group["Coordinates"][()]



# Some parameters:
galx,galy,galz=data[:,0],data[:,1],data[:,2]
galcen=[np.median(galx),np.median(galy),np.median(galz)]     # Galaxy center
mp=(1e11)/(len(galx))                                        # Particle mass
galcoords=[galx,galy,galz]



# Centering coordinates, radial distance to center and :
galcoords=np.array(galcoords) - np.resize(np.array(galcen),(3,1))
galdist = []
for i in range(len(galcoords[0])):
    d=np.sqrt((galcoords[0,i]**2)+(galcoords[1,i]**2)+(galcoords[2,i]**2))
    galdist.append(d)

    
    
# Density profile:
prof=[],[] 
Rbin=10
Rins=0
Rmax=np.amax(galdist)

for n in range(2,40):
    R1=Rins
    if (R1>Rmax):
        break
    R2=n*Rbin
    Npart=0
    for i in range(len(galdist)):            # Loop that checks for parwticles between both radius and counts them.
        if (galdist[i]>R1 and galdist[i]<R2):
            Npart+=1
    dens=Npart*mp/(1.25*pi*(R2**3-R1**3))
    prof[0].append(R1)
    prof[1].append(dens)
    Rins=R2
logprof=np.log10(prof)



# Fitting to NFW:
init_guesses=[10**8.5,10]
r=10**logprof[0]
popt,pcov=fit(func.logNFW,r,logprof[1],[10**8,10])



# Plotting radial profile and fit:
plt.figure(dpi=150)
plt.plot(logprof[0],logprof[1],'k.',ms=1.5)
plt.plot(logprof[0],func.logNFW(r,popt[0],popt[1]),'r-',alpha=0.5,linewidth=1)
plt.savefig("v0_NFWfit.PNG")
print("Final state:\n"+"rho_0 = "+str(popt[0])+"\n"+"log(rho_0) = "+str(np.log10(popt[0]))+"\nr_s = "+str(popt[1]))


#end

ValueError: array must not contain infs or NaNs

In [8]:
prof[1]

[2110394.5453985324,
 48249.0774847009,
 12388.27665147726,
 5009.467061253099,
 1399.1643348738053,
 1002.5508226261123,
 1054.75465244333,
 469.39706718347014,
 375.8640722465425,
 76.93290300514579,
 192.42915033780795,
 108.59185882602668,
 0.0,
 0.0,
 0.0,
 124.67461882351654,
 27.70923927606448,
 99.18126930751023,
 44.635917431556976,
 40.38824884171809,
 146.8769482030469,
 0.0,
 15.368008988957907,
 0.0,
 13.052173703077015,
 12.085804885953134,
 0.0,
 10.449237133649262,
 0.0,
 27.371684945936856,
 8.553843095298372,
 0.0,
 0.0,
 14.261994340354665,
 0.0,
 19.112927866927638,
 6.035740908912836,
 5.726285337239319]